# Topic Modeling of App Reviews

## Data Preparation

First, we load the CSV, filtering only the spanish comments and joining the "Problema" and "comment" fields, which are the fields that contain user-written text

We also remove empty and very small text inputs to avoid simple things as "me gusta"

In [157]:
import pandas as pd
import numpy as np

df = pd.read_csv("data.csv", encoding = "UTF-16", sep="\t")

df = df[df["Language"] == "es"]
df = df[["Problema", "comment"]]

df = df.replace(np.nan, '', regex=True)
df["text"] = [ x + y for x, y in zip(df["Problema"], df["comment"]) ]
df = df[(df["text"].str.strip() != "") & (df["text"].str.len() >= 25)]

print("Total: ", len(df["text"]))
print(df["text"][:5])

Total:  1204
2     No puedo modificar los datos personales. Mi ap...
3                   poder enviar tu tarjeta de embarque
8           Más interacción pero mejor que la anterior 
11                            no puedo crear un usuario
12    Está mucho mejor, es más fácil acceder a ver m...
Name: text, dtype: object


We need to import nltk to find the spanish stopwords, which are helper words such as "de", "la", "que" that we should ignore to not compromise our result.

You need to uncomment the nltk download line and download the spanish corpus first.

In [158]:
import nltk

# uncomment if you need this
# nltk.download() 

stop_words = nltk.corpus.stopwords.words('spanish')

print(stop_words[:10])

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se']


Now we remove those stop words and banned words from the reviews because they would only mess with the algorithm, also we remove the accents because some reviews use it, some don't

In [159]:
import unidecode

# banned words, we don't want to cluster around those
banned_words = ['aplicacion', 'app', 'anterior', 'actualizacion', 'actualizar', 'version']

text = [" ".join([unidecode.unidecode(word) for word in text.split(" ")
                  if word not in stop_words
                  and unidecode.unidecode(word) not in banned_words
                 ]) for text in df["text"]]

text[:10]

['No puedo modificar datos personales. Mi apellido incompleto. Soy CLAUDIA LEON PRADO ',
 'poder enviar tarjeta embarque',
 ' Mas interaccion mejor ',
 'puedo crear usuario',
 'Esta mejor, facil acceder ver kilometros. ',
 'En aparece opcion postular upgrade y, si postulando, caso, permite revisar listado postulacion. ',
 'Estoy tratando ver vuelo viaja esposo Lima Tacna saber si salio hora encuentro daba vuelo x dia dias anteriores ,posteriores.Ahora opcion...la pueden agregar xfavor ',
 ' Muy buena nueva ',
 'Que cuidado maletas  ',
 'No sirve comprar pasajes  ']

Next we tokennize and stemmatized words so we can get their essential meaning and avoid considering different genders or verb inflections of a word as a distinct word when they are probably talking about the same thing.

Those words are saved in a vocabulary for later.

In [160]:
from nltk.stem.snowball import SnowballStemmer
import re

stemmer = SnowballStemmer("spanish")

def tokenize(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

def stem(tokens):
    return [stemmer.stem(t) for t in tokens]

totalvocab_stemmed = []
totalvocab_tokenized = []
for i in text:
    allwords_tokenized = tokenize(i)
    totalvocab_tokenized.extend(allwords_tokenized)
    
    allwords_stemmed = stem(allwords_tokenized)
    totalvocab_stemmed.extend(allwords_stemmed)
    

vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')
print(vocab_frame.head())

there are 9224 items in vocab_frame
               words
no                no
pued           puedo
modific    modificar
dat            datos
personal  personales


## Fitting the Model

Now, we are going to send our text to a TfidfVectorizer, where we use IDF to make the algorithm focus on the main no-so-common words. We also build ngrams, so things like "check in" can be considered a single thing instead of two.

Then we send it to a Latent Dirichlet Allocation model, which should group the texts around clusters, discovering topics automatically for us.

In [161]:
%%time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import adjusted_rand_score


def tokenize_and_stem(text):
    return stem(tokenize(text))


vectorizer = TfidfVectorizer(max_df=0.6, max_features=200000,
                             min_df=5, stop_words=stop_words,
                             use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

X = vectorizer.fit_transform(text).todense()
print(X.shape)

terms = vectorizer.get_feature_names()
print(terms[90:100])

K = 7
model = LatentDirichletAllocation(n_components=K, max_iter=100, random_state=8)
model.fit(X)

model

(1204, 422)
['count', 'cre', 'cre usuari', 'cuand', 'cuelg', 'cuent', 'da', 'dan', 'dat', 'dat personal']


/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


CPU times: user 18.7 s, sys: 203 ms, total: 18.9 s
Wall time: 19.3 s


## Visualization

Here we use pyLDAvis which give us an off-the-shelf visualization for LDA. The topics with more reviews in it are the biggest circles.

You can click in any of it to see which are the most relevant words for that particular topic, and you can move the relevace slider to the right to see words that are more relevant to that topic and not others.

This is a great visualization because we can see 3 things:
- The biggest topics that we have to concern about
- Which topics have overlaps and which talks about completely distinct subjects
- Which words are in those topics, giving us a hint of what it is about

In [162]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(model, X, vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      22.739159        1       1 -0.000227 -0.052705
0      20.532122        1       2 -0.076565  0.134584
4      14.543113        1       3  0.063455  0.197967
6      12.994547        1       4  0.242823 -0.128898
3      11.270157        1       5 -0.207909 -0.147292
2      10.968426        1       6  0.032857 -0.020536
1       6.952475        1       7 -0.054434  0.016880, topic_info=     Category       Freq              Term      Total  loglift  logprob
term                                                                   
400   Default  41.000000               ver  41.000000  30.0000  30.0000
412   Default  94.000000              vuel  94.000000  29.0000  29.0000
402   Default  27.000000          ver vuel  27.000000  28.0000  28.0000
305   Default  23.000000          pued ver  23.000000  27.0000  27.0000
75    Default  24.000000             compr  24.000000  26.0000  26.0000
180   Default  19.000000            ingres  19.000000  25.0000  25.0000
243   Default  15.000000             pasaj  15.000000  24.0000  24.0000
306   Default  15.000000     pued ver vuel  15.000000  23.0000  23.0000
95    Default  22.000000             cuent  22.000000  22.0000  22.0000
10    Default  13.000000             agreg  13.000000  21.0000  21.0000
147   Default  11.000000              falt  11.000000  20.0000  20.0000
330   Default  29.000000            reserv  29.000000  19.0000  19.0000
153   Default  11.000000           funcion  11.000000  18.0000  18.0000
175   Default   8.000000             inclu   8.000000  17.0000  17.0000
408   Default  12.000000          visualiz  12.000000  16.0000  16.0000
45    Default  15.000000              buen  15.000000  15.0000  15.0000
77    Default  11.000000       compr pasaj  11.000000  14.0000  14.0000
3     Default  11.000000          actualiz  11.000000  13.0000  13.0000
287   Default  60.000000              pued  60.000000  12.0000  12.0000
227   Default   8.000000             numer   8.000000  11.0000  11.0000
389   Default  12.000000            usuari  12.000000  10.0000  10.0000
71    Default  12.000000               com  12.000000   9.0000   9.0000
301   Default  11.000000       pued ingres  11.000000   8.0000   8.0000
188   Default  12.000000          kilometr  12.000000   7.0000   7.0000
371   Default  15.000000            tarjet  15.000000   6.0000   6.0000
307   Default   8.000000     pued visualiz   8.000000   5.0000   5.0000
230   Default  30.000000            opcion  30.000000   4.0000   4.0000
143   Default   6.000000          excelent   6.000000   3.0000   3.0000
127   Default  13.000000           embarqu  13.000000   2.0000   2.0000
28    Default  13.000000            asient  13.000000   1.0000   1.0000
...       ...        ...               ...        ...      ...      ...
245    Topic7   3.774739  pasajer frecuent   4.514237   2.4872  -3.8996
324    Topic7   3.550372            reclam   4.289505   2.4770  -3.9609
365    Topic7   3.463536               tal   4.203028   2.4726  -3.9856
407    Topic7   3.447627       viaj realiz   4.187121   2.4717  -3.9902
80     Topic7   2.770309          consider   3.510047   2.4294  -4.2089
411    Topic7   3.207410              volv   4.303565   2.3721  -4.0624
14     Topic7   2.203159             algun   3.382048   2.2375  -4.4380
354    Topic7   2.174231               sid   3.371785   2.2273  -4.4512
370    Topic7   2.816358             tarif   4.711413   2.1515  -4.1925
224    Topic7   1.533452          noviembr   3.339595   1.8877  -4.8004
358    Topic7   2.726896              sirv   6.037905   1.8712  -4.2247
172    Topic7   1.233792             igual   2.755343   1.8626  -5.0178
142    Topic7   1.440562               etc   3.368198   1.8167  -4.8629
332    Topic7   0.978930           respond   2.513993   1.7229  -5.2492
323    Topic7   1.197621             recib   3.203091   1.6823  -5.0476
216    Topic7  

Of course we don't only want to see the chart about, we want to be able to effectively read the reviews in each topic, so, we build a dataframe with the original texts and the cluster number and score assigned to it. The score is the proximity of that review with the most revelant words in that cluster, so we filter those with less than 60% relevance because we only want the most focused ones.

In [163]:
df["text"][2]

result = model.transform(X)
clusters = [ np.argmax(score) for score in result ]
scores = [ max(score) for score in result ]

df["cluster"] = clusters
df["score"] = scores

df.loc[(df["score"] < 0.6), "cluster"] = K

df[["text","cluster","score"]].head()

,text,cluster,score
2,No puedo modificar los datos personales. Mi ap...,5,0.727286
3,poder enviar tu tarjeta de embarque,0,0.733866
8,Más interacción pero mejor que la anterior,5,0.644139
11,no puedo crear un usuario,4,0.705803
12,"Está mucho mejor, es más fácil acceder a ver m...",7,0.483479


Then we get the main words for each cluster, some examples and the sizes of the clusters

In [164]:
# copied from https://github.com/bmabey/pyLDAvis/blob/master/pyLDAvis/sklearn.py
def _get_topic_term_dists(lda_model):
    return _row_norm(lda_model.components_)

def _row_norm(dists):
    # row normalization function required
    # for doc_topic_dists and topic_term_dists
    return dists / dists.sum(axis=1)[:, None]

topic_term_dists = _get_topic_term_dists(model)

# sort cluster centers by proximity to centroid
order_centroids = topic_term_dists.argsort()[:, ::-1] 

cluster_words = [
    ", ".join([
        vocab_frame.loc[terms[ind].split(' ')].values.tolist()[0][0] for ind in order_centroids[i, :6]
    ]) for i in range(K)
]
cluster_words.append("Others")

print(cluster_words[0])

cluster_examples = [
    df[df["cluster"] == i]["text"].values.tolist() for i in range(len(cluster_words))
]

print(cluster_examples[0][0])

cluster_sizes = [ len(df[df["cluster"] == i]) for i in range(len(cluster_words)) ]

print(cluster_sizes)

hacer, tarjeta, embarque, asientos, in, check
poder enviar tu tarjeta de embarque
[149, 18, 54, 58, 97, 192, 82, 554]


And we print everything for you to read

In [165]:
data = { 'text': df["text"].values.tolist() }

frame = pd.DataFrame(data, index = [clusters], columns = ['text'])

print("Top terms per cluster:")
print()

for i in range(len(cluster_sizes)):
    print("Cluster %d total:" % i, cluster_sizes[i])
    print() #add whitespace
    
    print(cluster_words[i])
    print() #add whitespace
    
    print("Cluster %d examples:" % i)
    for title in cluster_examples[i][:15]:
        print('- %s' % title)
    print() #add whitespace
    print() #add whitespace

Top terms per cluster:

Cluster 0 total: 149

hacer, tarjeta, embarque, asientos, in, check

Cluster 0 examples:
- poder enviar tu tarjeta de embarque
- no asocia mi check in  y mi categoria de socio, no puedo elegir asientos, es un real problema
- La app anterior era mejor, en esta no hay donde hacer check in ni ver flight status de otros vuelos LATAM  
- No tengo la opción de check in 
- debería actualizar la aplicación anterior... porque tiene que ser otra? me pide datos y todo eso... una tontería!!!
- La debieron lanzar completamente funcional Parece que la actualización demora un mes en lo volado 
- no puedo entrar a mi cuenta
- La tarjeta de embarque no indica q soy Preferente  
- No se figuran todas las reservas. Cuando las añado, me dice que no puede y están trabajado en solucionarlo.  
- No puedo reenviar la tarjeta de embarque a mi jefe
- La tarjeta de embarque no indica la categoría de socio 
-  Todo súper, pero dónde está el botón de check in? 
- no pude iniciar sesion ya e

So, one problem with Unsupervised Learning of clustering is that you have to choose beforehand the number of clusters that you want, previously, we choose a small number, and maybe creating topics that are actually talking about two different things.

On the other hand, if we choose too many topics we might break those topics in smaller ones, but create many one which are actually talking about the same things.

We can visualize this effect on the chart, take a look with K = 30

In [166]:
import pyLDAvis
import pyLDAvis.sklearn
K2 = 30

model2 = LatentDirichletAllocation(n_components=K2, max_iter=100, random_state=0)
model2.fit(X)

pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(model2, X, vectorizer)

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
2      8.241454        1       1 -0.243206 -0.090169
4      7.923964        1       2 -0.146170 -0.012887
18     6.314597        1       3 -0.109888  0.063914
27     5.704464        1       4 -0.129658  0.167671
15     5.241829        1       5  0.009940  0.041214
19     5.204755        1       6 -0.133241 -0.055663
5      5.022071        1       7 -0.153425  0.127519
20     4.843267        1       8 -0.041895  0.056156
12     4.545650        1       9  0.048585 -0.067615
25     4.463536        1      10  0.015378  0.198974
13     3.948342        1      11 -0.137862  0.017280
24     3.425375        1      12 -0.004458 -0.138706
28     3.027199        1      13 -0.067729 -0.187833
14     3.024256        1      14 -0.005528 -0.003723
23     2.870744        1      15  0.087610 -0.005505
6      2.527001        1      16  0.030703  0.038666
29     2.396265        1      17  0.022535 -0.131716
11     2.236672        1      18  0.125861 -0.002299
17     2.193864        1      19 -0.012803 -0.035323
10     1.966099        1      20  0.088687  0.027743
9      1.941280        1      21  0.111000  0.069133
8      1.894942        1      22  0.023668 -0.061516
26     1.881305        1      23  0.026387 -0.057092
1      1.673514        1      24  0.086542 -0.008823
22     1.561936        1      25  0.090472  0.041021
16     1.481471        1      26  0.092697 -0.031498
7      1.432870        1      27  0.089306 -0.004749
21     1.003760        1      28  0.078830  0.015276
3      1.003760        1      29  0.078830  0.015276
0      1.003760        1      30  0.078830  0.015276, topic_info=     Category       Freq               Term      Total  loglift  logprob
term                                                                    
400   Default  37.000000                ver  37.000000  30.0000   30.000
386   Default  12.000000             upgrad  12.000000  29.0000   29.000
402   Default  24.000000           ver vuel  24.000000  28.0000   28.000
412   Default  88.000000               vuel  88.000000  27.0000   27.000
131   Default  17.000000           encuentr  17.000000  26.0000   26.000
406   Default  30.000000               viaj  30.000000  25.0000   25.000
28    Default  14.000000             asient  14.000000  24.0000   24.000
10    Default  14.000000              agreg  14.000000  23.0000   23.000
225   Default  18.000000               nuev  18.000000  22.0000   22.000
263   Default  15.000000                pod  15.000000  21.0000   21.000
371   Default  15.000000             tarjet  15.000000  20.0000   20.000
305   Default  21.000000           pued ver  21.000000  19.0000   19.000
180   Default  17.000000             ingres  17.000000  18.0000   18.000
38    Default  10.000000               bien  10.000000  17.0000   17.000
45    Default  16.000000               buen  16.000000  16.0000   16.000
71    Default  12.000000                com  12.000000  15.0000   15.000
75    Default  21.000000              compr  21.000000  14.0000   14.000
178   Default  14.000000        informacion  14.000000  13.0000   13.000
153   Default  12.000000            funcion  12.000000  12.0000   12.000
216   Default  10.000000             moment  10.000000  11.0000   11.000
174   Default  13.000000                 in  13.000000  10.0000   10.000
62    Default  13.000000              check  13.000000   9.0000    9.000
147   Default  12.000000               falt  12.000000   8.0000    8.000
351   Default  11.000000            servici  11.000000   7.0000    7.000
211   Default  15.000000              mejor  15.000000   6.0000    6.000
95    Default  21.000000              cuent  21.000000   5.0000    5.000
127   Default  13.000000            embarqu  13.000000   4.0000    4.000
63    Default  12.000000           check in  12.000000   3.0000    3.000
230   Default  28.000000             opcion  28.000000   2.0000    2.00

## Credits

I only managed to create this notebook thanks to those two awesome tutorials, which I basically copied for this topic modelling work:

1. Document Clustering with Python - http://brandonrose.org/clustering
2. Modern NLP in Python - https://www.youtube.com/watch?v=6zm9NC9uRkk

And thanks for the [@datasciencepython](https://web.telegram.org/#/im?p=@datasciencepython) group in telegram for poiting me in the right direction